In [0]:
########################
#Install Dependancies
########################
#Install openpyxl so can export as excel (only required to once for your computer)
%pip install openpyxl
%pip install pycountry
dbutils.library.restartPython()

In [0]:
########################
#Setup for Tranformation Tests
########################

#####################
#Define notebook setup
this_notebook_state = "paymentPending"

#Setup the Notebook Parameters (Defaulting to Payment Pending and Running all tests)
#Setup Test Reporting
from datetime import datetime, timedelta
run_user = spark.sql("SELECT current_user()").first()[0]
run_tag = "Testing Transformation Tests"
run_by_automation_name = "Transformation_tests"
#Capture Test Start datetime
run_start_datetime = datetime.now()
#####################

#Setup Notebook Parameters (Defaulting to Payment Pending and Running all tests)
dbutils.widgets.text("state_under_test", "")

#fields_to_exclude : Should be a comma separated list of fields to exclude 
dbutils.widgets.text("fields_to_exclude", "")

# Read parameters
state_under_test = dbutils.widgets.get("state_under_test")
fields_to_exclude = dbutils.widgets.get("fields_to_exclude")

#Set Default Values if not called from another Notebook
if state_under_test == "" :
    state_under_test = this_notebook_state

#Get Fields to Exclude
if fields_to_exclude != "":    
    fields_to_exclude = [item.strip() for item in fields_to_exclude.split(",")] 
else:
    fields_to_exclude = []
    
print(f"Fields to exclude = {str(fields_to_exclude)}")
print(f"Testing State = {state_under_test}")

#Import Tests
import Test_Functions.PaymentPending_Tests as pp_tests

#Import asdict to convert Dataclass to Dictionary
from dataclasses import asdict

import models
from models.test_result import TestResult

#Setup Global Variables
all_test_results = []
current_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
base_path = current_path.rsplit("/", 1)[0] + "/"
 #Below will be replaced eventually to store the reults in a spark table
test_results_path= "/Workspace/Users/peter.gresty@hmcts.net/Results/Transformation_Tests"



In [0]:
#Load Config and Setup Enviorment Variables
config = spark.read.option("multiline", "true").json("dbfs:/configs/config.json")
env_name = config.first()["env"].strip().lower()
lz_key = config.first()["lz_key"].strip().lower()
 
# print(f"env_code: {lz_key}")  # This won't be redacted
# print(f"env_name: {env_name}")  # This won't be redacted
 
KeyVault_name = f"ingest{lz_key}-meta002-{env_name}"
# print(f"KeyVault_name: {KeyVault_name}")
 
# Service principal credentials
client_id = dbutils.secrets.get(KeyVault_name, "SERVICE-PRINCIPLE-CLIENT-ID")
client_secret = dbutils.secrets.get(KeyVault_name, "SERVICE-PRINCIPLE-CLIENT-SECRET")
tenant_id = dbutils.secrets.get(KeyVault_name, "SERVICE-PRINCIPLE-TENANT-ID")
 
# Storage account names
curated_storage = f"ingest{lz_key}curated{env_name}"
checkpoint_storage = f"ingest{lz_key}xcutting{env_name}"
raw_storage = f"ingest{lz_key}raw{env_name}"
landing_storage = f"ingest{lz_key}landing{env_name}"
external_storage = f"ingest{lz_key}external{env_name}"
  
# Spark config for curated storage (Delta table)
spark.conf.set(f"fs.azure.account.auth.type.{curated_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{curated_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{curated_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{curated_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{curated_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")
 
# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{checkpoint_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{checkpoint_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{checkpoint_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{checkpoint_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{checkpoint_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")
 
# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{raw_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{raw_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{raw_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{raw_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{raw_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")
 
# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{landing_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{landing_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{landing_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{landing_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{landing_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")
 
 
# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{external_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{external_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{external_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{external_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{external_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")
  
# Setting variables for use in subsequent cells
bronze_path = f"abfss://bronze@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS/"
silver_path = f"abfss://silver@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS/"
audit_path = f"abfss://silver@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS/AUDIT/{state_under_test}"
gold_path = f"abfss://gold@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS/{state_under_test}"
 
 
# Print all variables
# variables = {
#     # "read_hive": read_hive,
    
#     "bronze_path": bronze_path,
#     "silver_path": silver_path,
#     "audit_path": audit_path,
#     "gold_path": gold_path,
#     "key_vault": KeyVault_name,
#     "AppealState": state_under_test
 
# }
 
# display(variables)

import json

#Get Latest Json Folder
json_location = dbutils.fs.ls(f"{gold_path}/")[-1]
latest_json_location = json_location.name
dbutils.fs.ls(f"{gold_path}/{latest_json_location}")

#Set Paths
try: 
    json_path = f"{gold_path}/{latest_json_location}/JSON/"
    # json_path = f"{gold_path}/{latest_json_location}/INVALID_JSON/"
    M1_silver = f"{silver_path}/silver_appealcase_detail"
    M1_bronze = f"{bronze_path}/bronze_appealcase_crep_rep_floc_cspon_cfs"
    M2_bronze = f"{bronze_path}/bronze_appealcase_caseappellant_appellant"
    M3_bronze = f"{bronze_path}/bronze_status_htype_clist_list_ltype_court_lsitting_adj"
    M2_silver = f"{silver_path}/silver_caseapplicant_detail"
    M3_silver = f"{silver_path}/silver_status_detail"
    C = f"{silver_path}/silver_appealcategory_detail"
    bhc = f"{bronze_path}/bronze_hearing_centres"
    bat = f"{bronze_path}/bronze_appealtype" 
    docsr = f"{bronze_path}/bronze_documentsreceived"
    apl_audit = f"{audit_path}/apl_active_payment_pending_cr_audit_table/"   
    H_bronze = f"{bronze_path}/bronze_history"
    H_silver = f"{silver_path}/silver_history_detail"
except:
    print(f"Error during fetch: {str(e)}")

#Create and Load Dataframes
json_data = spark.read.format("json").load(json_path)
M1_silver = spark.read.format("delta").load(M1_silver)
M1_bronze = spark.read.format("delta").load(M1_bronze)
M2_bronze = spark.read.format("delta").load(M2_bronze)
M2_silver = spark.read.format("delta").load(M2_silver)
M3_silver = spark.read.format("delta").load(M3_silver)
M3_bronze = spark.read.format("delta").load(M3_bronze)
C = spark.read.format("delta").load(C)
bhc = spark.read.format("delta").load(bhc)
bat = spark.read.format("delta").load(bat)
docsr = spark.read.format("delta").load(docsr)
apl_audit = spark.read.format("delta").load(apl_audit)
H_bronze = spark.read.format("delta").load(H_bronze)
H_silver = spark.read.format("delta").load(H_silver)

#Can be removed later, added to allow developing of code in this notebook to begin with before moving to func files
from pyspark.sql.functions import (
    col, when, lit, array, struct, collect_list, 
    max as spark_max, date_format, row_number, expr, 
    size, udf, coalesce, concat_ws, concat, trim, year, split, datediff,
    collect_set, current_timestamp,transform, first, array_contains
)

#After Setup

In [0]:
#Used to Quickly reload the function file containing the test, rather than restart python
# import importlib
# importlib.reload(pp_tests)

# appealType

- appealType
- hmctsCaseCategory
- appealTypeDescription
- appealReferenceNumber
- caseManagementCategory

In [0]:
# This only seems to work with 'invalid_json'. This is because CCDAppealType is in the invalid JSON but not the valid.

#Init data for test
test_data_setup = None
test_df_appealType, test_data_setup =  pp_tests.test_appealType_init(json_data, M1_silver, C, bat)
if test_data_setup != True:
     all_test_results.append(test_data_setup)


if test_df_appealType != None:
     check_fields = ["appealType", "hmctsCaseCategory", "appealTypeDescription"]
     if not all(field in fields_to_exclude for field in check_fields):
          all_test_results.append(pp_tests.test_appealType_ac1(test_df_appealType))
          all_test_results.append(pp_tests.test_appealType_ac3(test_df_appealType))
          all_test_results.append(pp_tests.test_appealType_ac3(test_df_appealType))
          all_test_results.append(pp_tests.test_appealType_ac4(test_df_appealType))
          all_test_results.append(pp_tests.test_appealType_ac5(test_df_appealType))
          all_test_results.append(pp_tests.test_appealType_ac6(test_df_appealType))
          all_test_results.append(pp_tests.test_appealType_ac7(test_df_appealType))
          all_test_results.append(pp_tests.test_appealType_ac8(test_df_appealType))
          all_test_results.append(pp_tests.test_appealType_ac9(test_df_appealType))
          all_test_results.append(pp_tests.test_appealType_ac10(test_df_appealType))
          all_test_results.append(pp_tests.test_appealType_ac11(test_df_appealType))
          all_test_results.append(pp_tests.test_appealType_ac12(test_df_appealType))
          all_test_results.append(pp_tests.test_appealType_ac13(test_df_appealType))

     if "appealReferenceNumber" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_appealReferenceNumber_ac1(test_df_appealType))

# caseData

In [0]:
#Init data for test
test_data_setup = None
test_df, test_data_setup =  pp_tests.test_caseData_init(json_data, M1_bronze, M1_silver)
if test_data_setup != True:
     all_test_results.append(test_data_setup)

if test_df != None:
     if "submissionOutOfTime" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_submissionOutOfTime_ac1(test_df))
          all_test_results.append(pp_tests.test_submissionOutOfTime_ac2(test_df))
          all_test_results.append(pp_tests.test_submissionOutOfTime_ac3(test_df))
                                                                              
     if "appealSubmissionDate" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_appealSubmissionDate(test_df))

     if "appealSubmissionInternalDate" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_appealSubmissionInternalDate(test_df))
     
     if "tribunalReceivedDate" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_tribunalReceivedDate(test_df))
            
     if "appellantsRepresentation" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_appellantsRepresentation_ac1(test_df))
          all_test_results.append(pp_tests.test_appellantsRepresentation_ac2(test_df))

     if "recordedOutOfTimeDecision" not in fields_to_exclude:
          rod_test_df, init_success = pp_tests.test_recordedOutOfTimeDecision_init(json_data, M1_bronze, M3_bronze)

          if init_success is not True:
               all_test_results.append(rod_test_df) 
          else:
               all_test_results.append(pp_tests.test_recordedOutOfTimeDecision_ac1(rod_test_df))
               all_test_results.append(pp_tests.test_recordedOutOfTimeDecision_ac2(rod_test_df))
               all_test_results.append(pp_tests.test_recordedOutOfTimeDecision_ac3(rod_test_df))

# hearingCentre

In [0]:
check_fields = ["hearingCentre", "staffLocation", "caseManagementLocation", "selectedHearingCentreRefData", "applicationChangeDesignatedHearingCentre"]
if not all(field in fields_to_exclude for field in check_fields):
    all_test_results.extend(pp_tests.hearing_centre_field_test(M1_silver, M2_silver, H_silver, bhc, json_data, state_under_test))

# flagLabels

In [0]:
#Init data for test
test_data_setup = None
test_df, test_data_setup =  pp_tests.test_flags_init(json_data, M1_bronze, C)
if test_data_setup != True:
     all_test_results.append(test_data_setup)

if test_df != None:
     if "caseFlags" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_caseFlags(test_df))
     
     if "appellantLevelFlags" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_appellantFlags(test_df))

     if "isAriaMigratedFeeExemption" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_isAriaMigratedFeeExemption_ac1(test_df))
          all_test_results.append(pp_tests.test_isAriaMigratedFeeExemption_ac2(test_df))

# appellantDetails

In [0]:
#Init data for test
test_data_setup = None
test_df, test_data_setup =  pp_tests.test_appellantdetails_init(json_data, M2_bronze, M1_bronze, C)
if test_data_setup != True:
     all_test_results.append(test_data_setup)

if test_df != None:
     if "appellantFamilyName" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_appellantFamilyName(test_df))

     if "appellantGivenNames" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_appellantGivenNames(test_df))

     if "appellantNameForDisplay" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_appellantNameForDisplay(test_df))

     if "appellantDateOfBirth" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_appellantDateOfBirth(test_df))

     if "isAppellantMinor" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_isAppellantMinor(test_df))

     if "caseNameHmctsInternal" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_caseNameHmctsInternal(test_df))

     if "hmctsCaseNameInternal" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_hmctsCaseNameInternal(test_df))

     if "internalAppellantEmail" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_internalAppellantEmail_ac1(test_df))
          all_test_results.append(pp_tests.test_internalAppellantEmail_ac2(test_df))

     if "email" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_email_ac1(test_df))
          all_test_results.append(pp_tests.test_email_ac2(test_df))

     if "internalAppellantMobileNumber" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_internalAppellantMobileNumber_ac1(json_data, M2_bronze))
          all_test_results.append(pp_tests.test_internalAppellantMobileNumber_ac2(json_data, M2_bronze))

     if "mobileNumber" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_mobileNumber_ac1(json_data, M2_bronze))
          all_test_results.append(pp_tests.test_mobileNumber_ac2(json_data, M2_bronze))

     if "appellantInUk" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_appellantInUk_ac1(test_df))
          all_test_results.append(pp_tests.test_appellantInUk_ac2(test_df))

     if "appealOutOfCountry" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_appealOutOfCountry_ac1(test_df))
          all_test_results.append(pp_tests.test_appealOutOfCountry_ac2(test_df))

     if "oocAppealAdminJ" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_oocAppealAdminJ_ac1(test_df))
          all_test_results.append(pp_tests.test_oocAppealAdminJ_ac2(test_df))
          all_test_results.append(pp_tests.test_oocAppealAdminJ_ac3(test_df))
          all_test_results.append(pp_tests.test_oocAppealAdminJ_ac4(test_df))
          all_test_results.append(pp_tests.test_oocAppealAdminJ_ac5(test_df))

     if "appellantAddress" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_appellantAddress_ac1(test_df))
          all_test_results.append(pp_tests.test_appellantAddress_ac2(test_df))

     if "addressLine1AdminJ" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_addressLine1AdminJ_ac1(test_df))
          all_test_results.append(pp_tests.test_addressLine1AdminJ_ac2(test_df))

     if "addressLine2AdminJ" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_addressLine2AdminJ_ac1(test_df))
          all_test_results.append(pp_tests.test_addressLine2AdminJ_ac2(test_df))

     if "addressLine3AdminJ" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_addressLine3AdminJ_ac1(test_df))
          all_test_results.append(pp_tests.test_addressLine3AdminJ_ac2(test_df))

     if "addressLine4AdminJ" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_addressLine4AdminJ_ac1(test_df))
          all_test_results.append(pp_tests.test_addressLine4AdminJ_ac2(test_df))

     if "countryGovUkOocAdminJ" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_countryGovUkOocAdminJ_ac1(test_df))
          all_test_results.append(pp_tests.test_countryGovUkOocAdminJ_ac2(test_df,external_storage,spark)[0])

     if "appellantStateless" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_appellantStateless_ac1(test_df))
          all_test_results.append(pp_tests.test_appellantStateless_ac2(test_df))

     if "appellantNationalities" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_appellantNationalities(test_df))

     if "appellantNationalitiesDescription" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_appellantNationalitiesDescription(test_df))

     if "deportationOrderOptions" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_deportationOrderOptions_ac1(test_df))
          all_test_results.append(pp_tests.test_deportationOrderOptions_ac2(test_df))
          all_test_results.append(pp_tests.test_deportationOrderOptions_ac3(test_df))
          all_test_results.append(pp_tests.test_deportationOrderOptions_ac3(test_df))

# legalRepDetails

In [0]:
#Init data for test
test_data_setup = None
test_df, test_data_setup =  pp_tests.test_legalRepDetails_init(json_data, M1_bronze, M1_silver)
if test_data_setup != True:
     all_test_results.append(test_data_setup)


if test_df != None:
     #legalRepEmail
     #Test1 - legalRepEmail - Representation = AIP = Ommited
     if "legalRepEmail" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_legalRepEmail_test1(test_df))

          
     # legalRepEmail - If legalRepEmail != M1.RepEmail or M1.CaseRepEmail or M1.FileSpecificEmail
     #Test2 - RepEmail has data so expect legalRepEmail = RepEmail
     if "legalRepEmail" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_legalRepEmail_test2(test_df))

     #Test3 - RepEmail has NO data but CaseRepEmail Does so expect legalRepEmail = CaseRepEmail
     if "legalRepEmail" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_legalRepEmail_test3(test_df))
     
     #Test4 - RepEmail, CaseRepEmail have no data so expect legalRepEmail = FileSpecificEmail
     if "legalRepEmail" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_legalRepEmail_test4(test_df))

     #Test5 - Check Cleaning of Email address is correct
     if "legalRepEmail" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_legalRepEmail_test5(test_df))
     
     #legalRepGivenName - IF Contact IS NULL USE RepName; IF RepName IS NULL USE CaseRepName

     #test1 - Contact is NOT NULL - legalRepGivenName = Contact
     if "legalRepGivenName" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_legalRepGivenName_test1(test_df))
     
     #test2 - Contact is NULL -RepName is not null --  legalRepGivenName = RepName
     if "legalRepGivenName" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_legalRepGivenName_test2(test_df))

     #test3 - Contact is NULL -RepName is null --  legalRepGivenName = CaseRepName
     if "legalRepGivenName" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_legalRepGivenName_test3(test_df))

     # #legalRepFamilyNamePaperJ - IF RepName IS NULL USE CaseRepName          

     #Test1 - RepName is not null -- legalRepFamilyNamePaperJ = RepName     
     if "legalRepFamilyNamePaperJ" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_legalRepFamilyNamePaperJ_test1(test_df))
     
     #Test2 - RepName is null -- legalRepFamilyNamePaperJ = CaseRepName
     if "legalRepFamilyNamePaperJ" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_legalRepFamilyNamePaperJ_test2(test_df))

     #legalRepCompanyPaperJ - IF RepName IS NULL USE CaseRepName     

     #Test1 - Rep_Name is not null -- legalRepCompanyPaperJ = RepName     
     if "legalRepCompanyPaperJ" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_legalRepCompanyPaperJ_test1(test_df))
     
     #Test2 - Rep_Name is null -- legalRepCompanyPaperJ = CaseRepName     
     if "legalRepCompanyPaperJ" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_legalRepCompanyPaperJ_test2(test_df))

     #legalRepHasAddress
     #Test1 - RepId >0 -- legalRepHasAddress = Yes
     if "legalRepHasAddress" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_legalRepHasAddress_test1(test_df))

     #NOTE : No data for test below
     #test2 -  RepId = 0 and Rep_Postcode is uk postcode legalRepHasAddress = yes
     if "legalRepHasAddress" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_legalRepHasAddress_test2(test_df))
     
     #NOTE : No data for test below - look into seeding from gold
     #test3 -  RepId = 0 and Rep_Postcode is NOT uk postcode legalRepHasAddress = nos
     if "legalRepHasAddress" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_legalRepHasAddress_test3(test_df))

     #legalRepAddressUK
     #Test1 - M1.RepresentativeId > 0 - combines Rep address fields
     if "legalRepAddressUK" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_legalRepAddressUK_test1(test_df))
     
     #Test2 - M1.RepresentativeId =0 - combines CaseRep address fields
     if "legalRepAddressUK" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_legalRepAddressUK_test2(test_df))

     
     #oocAddressLine1
     #NOTE LOW COVERAGE on oocAddressLine1,2,3,4 : To test when oocAddressLine1 is null and that oocAddressLine2 is used etc etc, we need data in the right state which we currently dont have. Look at creating data
     
     #Test1 - legalRepHasAddress =Yes Then oocaddress1 = Null
     if "oocAddressLine1" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_oocAddressLine1_test1(test_df))
      
      #Test2 - legalRepHasAddress =No Then oocaddress1 = CaseRepAddress1
     if "oocAddressLine1" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_oocAddressLine1_test2(test_df))


     #oocAddressLine2     
     #Test1 - legalRepHasAddress =Yes Then oocaddress1 = Null
     if "oocAddressLine2" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_oocAddressLine2_test1(test_df))
      
      #Test2 - legalRepHasAddress =No Then oocaddress1 = CaseRepAddress1
     if "oocAddressLine2" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_oocAddressLine2_test2(test_df))
     
     #oocAddressLine3          
     #Test1 - legalRepHasAddress =Yes Then oocaddress1 = Null
     if "oocAddressLine3" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_oocAddressLine3_test1(test_df))
      
      #Test2 - legalRepHasAddress =No Then oocaddress1 = CaseRepAddress1
     if "oocAddressLine3" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_oocAddressLine3_test2(test_df))

     #oocAddressLine4 
     #FAILED - ARIA-DM-1753
     if "oocAddressLine4" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_oocAddressLine4_test1(test_df))
      
      #Test2 - legalRepHasAddress =No Then oocaddress1 = CaseRepAddress1
     if "oocAddressLine4" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_oocAddressLine4_test2(test_df))
       
     #oocLrCountryGovUkAdminJ
     #Test 1 - Check all records have expected country code
     if "oocLrCountryGovUkAdminJ" not in fields_to_exclude:          
          result, df = pp_tests.test_oocLrCountryGovUkAdminJ_test1(test_df,external_storage,spark)
          all_test_results.append(result)
          # display(df.select("oocLrCountryGovUkAdminJ", "countrycode","countryresult"))

# sponsorDetails 

In [0]:
#Init data for test
test_data_setup = None
test_df_sd, test_data_setup =  pp_tests.test_sponsorDetails_init(json_data, M1_bronze, C)
if test_data_setup != True:
     all_test_results.append(test_data_setup)
# display(test_df_808)

if test_df_sd != None:
   if "hasSponsor" not in fields_to_exclude:
      #Acceptance Criteria - hasSponsor - "IF CategoryId IN [38] = Include; ELSE OMIT, IF SponsorName IS NOT NULL = Yes; ELSE No"
            
      #test1 - IF CategoryId not in 38 and hasSponsor not omitted 
      all_test_results.append(pp_tests.test_hasSponsor_test1(test_df_sd))

      #test2 - IF CategoryId IN [38] + Sponsor_Name is Null and hasSponsor != No
      all_test_results.append(pp_tests.test_hasSponsor_test2(test_df_sd))

      #test3 - IF CategoryId IN [38] + Sponsor_Name is not Null and hasSponsor != Yes
      all_test_results.append(pp_tests.test_hasSponsor_test3(test_df_sd))

   if "sponsorGivenNames" not in fields_to_exclude:
      # Acceptance Criteria - sponsorGivenNames - IF CategoryId IN [38] = Include; ELSE OMIT, IF SponsorName IS NOT NULL = Include; ELSE OMIT

      #test1 - IF CategoryId not in 38 and sponsorGivenNames not omitted 
      all_test_results.append(pp_tests.test_sponsorGivenNames_test1(test_df_sd))
      
      #test2 - If CategoryId in 38 + Sponsor_Name is Null and sponsorGivenNames is not omitted
      all_test_results.append(pp_tests.test_sponsorGivenNames_test2(test_df_sd))

      #BUG: ARIADM-1733 (no longer a bug - incorrect test)
      #test3 - If CategoryId in 38 + Sponsor_Name is not Null and sponsorGivenNames is omitted
      all_test_results.append(pp_tests.test_sponsorGivenNames_test3(test_df_sd))

   if "sponsorFamilyName" not in fields_to_exclude:
      # Acceptance Criteria - sponsorFamilyName - IF CategoryId IN [38] = Include; ELSE OMIT, IF SponsorName IS NOT NULL = Include; ELSE OMIT

      #test1 - If CategoryId not in 38 and sponsorFamilyName not omitted
      all_test_results.append(pp_tests.test_sponsorFamilyName_test1(test_df_sd))
      
      #test2 - If CategoryId in 38 + Sponsor_Name is Null and sponsorFamilyName is not omitted
      all_test_results.append(pp_tests.test_sponsorFamilyName_test2(test_df_sd))

      #test3 - If CategoryId in 38 + Sponsor_Name is not Null and sponsorFamilyName is omitted
      all_test_results.append(pp_tests.test_sponsorFamilyName_test3(test_df_sd))

   if "sponsorAuthorisation" not in fields_to_exclude:
      # Acceptance Criteria 4 - sponsorAuthorisation - IF CategoryId IN [38] = Include; ELSE OMIT, IF SponsorName IS NOT NULL = Include; ELSE OMIT, IF Sponsor_Authorised IS 1 = Yes; ELSE No

      #BUG: ARIADM-1734 
      #test1 - If CategoryId not in 38 and sponsorAuthorisation is not omitted
      all_test_results.append(pp_tests.test_sponsorAuthorisation_test1(test_df_sd))

      #test2 - IF CategoryId IN [38] + Sponsor_Authorised IS 1 + Sponsor_Name is not null and sponsorAuthorisation != Yes
      all_test_results.append(pp_tests.test_sponsorAuthorisation_test2(test_df_sd))

      #test3 - IF CategoryId IN [38] + Sponsor_Authorised IS 0 + Sponsor_Name is not null and sponsorAuthorisation != No
      all_test_results.append(pp_tests.test_sponsorAuthorisation_test3(test_df_sd))

      #BUG: ARIADM-1735 
      #test4 - IF CategoryId IN [38] + SponsorName IS NULL and sponsorAuthorisation is omitted
      all_test_results.append(pp_tests.test_sponsorAuthorisation_test4(test_df_sd))

   if "sponsorAddress" not in fields_to_exclude:
      # Acceptance Critera - sponsorAddress - IF CategoryId IN [38] = Include; ELSE OMIT, 
      # sponsorAddress collection is mapped using ARIA Sponsor_Address1, Sponsor_Address2, Sponsor_Address3, Sponsor_Address4, Sponsor_Address5, Sponsor_Postcode as per in the mapping document APPENDIX-Address

      #test1 - If CategoryId is not 38 and sponsorAddress has not been omitted
      all_test_results.append(pp_tests.test_sponsorAddress_ac1(test_df_sd))

      #test2 - If CategoryId is 38 + Sponsor_Name is null and sponsorEmailAdminJ not omitted
      all_test_results.append(pp_tests.test_sponsorAddress_ac2(test_df_sd))

      #test3 - If CategoryId in 38 + Sponsor_Name is not Null and sponsorAddress is omitted
      all_test_results.append(pp_tests.test_sponsorAddress_ac3(test_df_sd))

      #test4 - Check sponsorAddress is concatenated correctly
      all_test_results.append(pp_tests.test_sponsorAddress_ac4(test_df_sd))

   if "sponsorEmailAdminJ" not in fields_to_exclude:
      #test1 - If CategoryId not 38 and sponsorEmailAdminJ not omitted
      all_test_results.append(pp_tests.test_sponsorEmailAdminJ_ac1(test_df_sd))
      #test2 - If CategoryId is 38 + Sponsor_Name is null and sponsorEmailAdminJ not omitted
      all_test_results.append(pp_tests.test_sponsorEmailAdminJ_ac2(test_df_sd))
      #test3 - If CategoryId is 38 + Sponsor_Name is not null and sponsorEmailAdminJ omitted
      all_test_results.append(pp_tests.test_sponsorEmailAdminJ_ac3(test_df_sd))
      #test4 - Check fields have been cleansed correctly
      all_test_results.append(pp_tests.test_sponsorEmailAdminJ_ac4(test_df_sd))

   if "sponsorMobileNumberAdminJ" not in fields_to_exclude:
      #test1 - If CategoryId not 38 and sponsorMobileNumberAdminJ not omitted
      all_test_results.append(pp_tests.test_sponsorMobileNumberAdminJ_ac1(test_df_sd))
      #test2 - If CategoryId is 38 + Sponsor_Name is null and sponsorMobileNumberAdminJ not omitted
      all_test_results.append(pp_tests.test_sponsorMobileNumberAdminJ_ac2(test_df_sd))
      #test3 - If CategoryId is 38 + Sponsor_Name is not null and sponsorMobileNumberAdminJ omitted
      all_test_results.append(pp_tests.test_sponsorMobileNumberAdminJ_ac3(test_df_sd))
      #test4 - Check fields have been cleansed correctly
      all_test_results.append(pp_tests.test_sponsorMobileNumberAdminJ_ac4(test_df_sd))

# partyIds

In [0]:
#Init data for test
test_data_setup = None
test_df, test_data_setup =  pp_tests.test_partyIds_init(json_data, M1_silver)
if test_data_setup != True:
     all_test_results.append(test_data_setup)

if test_df != None:
    if "appellantPartyId" not in fields_to_exclude:
        all_test_results.append(pp_tests.test_appellantPartyId(test_df))

    if "legalRepIndividualPartyId" not in fields_to_exclude:
        all_test_results.append(pp_tests.test_legalRepIndividualPartyId_ac1(test_df))
        all_test_results.append(pp_tests.test_legalRepIndividualPartyId_ac2(test_df))

    if "legalRepOrganisationPartyId" not in fields_to_exclude:
        all_test_results.append(pp_tests.test_legalRepOrganisationPartyId_ac1(test_df))
        all_test_results.append(pp_tests.test_legalRepOrganisationPartyId_ac2(test_df))

    if "sponsorPartyId" not in fields_to_exclude:
        all_test_results.append(pp_tests.test_sponsorPartyId_ac1(test_df))
        all_test_results.append(pp_tests.test_sponsorPartyId_ac2(test_df))

# payment

In [0]:
#Init data for test
test_data_setup = None
test_df, test_data_setup =  pp_tests.test_payment_init(json_data, M1_bronze)
if test_data_setup != True:
     all_test_results.append(test_data_setup)

if test_df != None:
     if "feeAmountGbp" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_feeAmountGbp_ac1(test_df))
          all_test_results.append(pp_tests.test_feeAmountGbp_ac2(test_df))

     if "feeDescription" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_feeDescription_ac1(test_df))
          all_test_results.append(pp_tests.test_feeDescription_ac2(test_df))

     if "feeWithHearing" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_feeWithHearing_ac1(test_df))
          all_test_results.append(pp_tests.test_feeWithHearing_ac2(test_df))

     if "feeWithoutHearing" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_feeWithoutHearing_ac1(test_df))
          all_test_results.append(pp_tests.test_feeWithoutHearing_ac2(test_df))

     if "paymentDescription" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_paymentDescription_ac1(test_df))
          all_test_results.append(pp_tests.test_paymentDescription_ac2(test_df))

     if "decisionHearingFeeOption" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_decisionHearingFeeOption_ac1(test_df))
          all_test_results.append(pp_tests.test_decisionHearingFeeOption_ac2(test_df))

# remission

In [0]:
#Init data for test
test_data_setup = None
test_df, test_data_setup =  pp_tests.test_remission_init(json_data, M1_bronze)
if test_data_setup != True:
     all_test_results.append(test_data_setup)

if test_df != None:
     check_fields = ["remissionType", "remissionClaim", "feeRemissionType", "exceptionalCircumstances", "legalAidAccountNumber", "asylumSupportReference", "helpWithFeesReferenceNumber"]
     if not all(field in fields_to_exclude for field in check_fields):
          all_test_results.append(pp_tests.test_remission_ac1(test_df))
          all_test_results.append(pp_tests.test_remission_ac2(test_df))
          all_test_results.append(pp_tests.test_remission_ac3(test_df))
          all_test_results.append(pp_tests.test_remission_ac4(test_df))
          all_test_results.append(pp_tests.test_remission_ac5(test_df))
          all_test_results.append(pp_tests.test_remission_ac6(test_df))
          all_test_results.append(pp_tests.test_remission_ac7(test_df))
          all_test_results.append(pp_tests.test_remission_ac8(test_df))
          all_test_results.append(pp_tests.test_remission_ac9(test_df))
          all_test_results.append(pp_tests.test_remission_ac10(test_df))
          all_test_results.append(pp_tests.test_remission_ac11(test_df))
          all_test_results.append(pp_tests.test_remission_ac12(test_df))

# homeOffice

In [0]:
#Init data for test
test_data_setup = None
test_df, test_data_setup =  pp_tests.test_homeOffice_init(json_data, C, M1_bronze, M2_bronze)
if test_data_setup != True:
     all_test_results.append(test_data_setup)

if test_df != None:
     if "homeOfficeDecisionDate" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_homeOfficeDecisionDate_ac1(test_df))
          all_test_results.append(pp_tests.test_homeOfficeDecisionDate_ac2(test_df))

     if "decisionLetterReceivedDate" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_decisionLetterReceivedDate_ac1(test_df))
          all_test_results.append(pp_tests.test_decisionLetterReceivedDate_ac2(test_df))
          all_test_results.append(pp_tests.test_decisionLetterReceivedDate_ac3(test_df))


     if "dateEntryClearanceDecision" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_dateEntryClearanceDecision_ac1(test_df))
          all_test_results.append(pp_tests.test_dateEntryClearanceDecision_ac2(test_df))
          all_test_results.append(pp_tests.test_dateEntryClearanceDecision_ac3(test_df))

     if "gwfReferenceNumber" not in fields_to_exclude:
          all_test_results.append(pp_tests.test_gwfReferenceNumber_ac1(test_df))
          all_test_results.append(pp_tests.test_gwfReferenceNumber_ac2(test_df))
          #Failure due to obfuscation  
          all_test_results.append(pp_tests.test_gwfReferenceNumber_ac3(test_df))

     if "homeOfficeReferenceNumber" not in fields_to_exclude:
          ho_test_df, init_success = pp_tests.test_homeOfficeReferenceNumber_init(json_data, C, M1_bronze, M2_bronze)

          if init_success is not True:
               all_test_results.append(ho_test_df) 
          else:
               all_test_results.append(pp_tests.test_homeOfficeReferenceNumber_ac1(ho_test_df))
               all_test_results.append(pp_tests.test_homeOfficeReferenceNumber_ac2(ho_test_df))
               all_test_results.append(pp_tests.test_homeOfficeReferenceNumber_ac3(ho_test_df))

# general

In [0]:
#Init data for test
test_data_setup = None
test_df, test_data_setup =  pp_tests.test_isServiceRequestTabVisibleConsideringRemissions_init(json_data, M1_bronze)
if test_data_setup != True:
     all_test_results.append(test_data_setup)

if test_df != None:
    if "isServiceRequestTabVisibleConsideringRemissions" not in fields_to_exclude:
        all_test_results.append(pp_tests.test_isServiceRequestTabVisibleConsideringRemissions_ac1(test_df))
        all_test_results.append(pp_tests.test_isServiceRequestTabVisibleConsideringRemissions_ac2(test_df))
        all_test_results.append(pp_tests.test_isServiceRequestTabVisibleConsideringRemissions_ac3(test_df))
        all_test_results.append(pp_tests.test_isServiceRequestTabVisibleConsideringRemissions_ac4(test_df))

# Default mappings

In [0]:
test_data_setup = None
test_df, test_data_setup =  pp_tests.test_default_mapping_init(json_data)
if test_data_setup != True:
     all_test_results.append(test_data_setup)

if test_df != None:
    all_test_results.append(pp_tests.test_defaultValues(test_df))

# Results processing

In [0]:
#######################
#PROCESS RESULTS
#######################

#Convert Results Classes to string
all_test_results_string = json.dumps([asdict(r) for r in all_test_results])

#Print Results
pass_count = sum(1 for r in all_test_results if r.status.upper() == "PASS")
fail_count = sum(1 for r in all_test_results if r.status.upper() == "FAIL")
print(f"TEST RESULTS FOR STATE : {state_under_test} -- Pass: {pass_count}, Fail: {fail_count}")

#If PaymentPending output report, Else exit and send results to Parent Notebook
notebook_path = base_path + "Shared_Notebooks/Common_Produce_TestResults"

try:
    #Check If if this notebook is parent notebook(so produce report), or has been called by another(return data to parent)
    if state_under_test == this_notebook_state:
        #Call Print Test Report Notebook
        dbutils.notebook.run( notebook_path, 600, {
        "all_test_results_string":all_test_results_string,
        "state_under_test":state_under_test,
        "base_path":base_path,
        "test_results_path": test_results_path,
        "run_user":run_user,
        "run_tag":run_tag,
        "run_by_automation_name": run_by_automation_name,
        "run_start_datetime": run_start_datetime.strftime("%Y-%m-%d %H:%M:%S"),       
        })    
    #else return to parent notebook
    else:
        print(f"Exiting Notebook : {this_notebook_state} and returning Data to Parent notebook : {state_under_test}")
        dbutils.notebook.exit(all_test_results_string) 

except Exception as e:
    print(f"Failed to run notebook at {notebook_path}. Check if the path exists and you have permission. Error: {e}")
    dbutils.notebook.exit(all_test_results_string)

